##  复现 Aiygari Model

### 复现了Aiygari Model的主要实验结果，并进一步测试了在不同的 (theta, delta) 条件下的模型求解结果

### Table of Contents  <!-- omit in toc -->

- [Aiyagari Model Realization](#realization)
- [Main Results](#main)
- [More Cases](#cases)

### Aiyagari Model Realization

In [16]:
import quantecon as qe  
import math
import numpy as np
import copy

# 初始化超参数
beta = 0.96  
mu = 3  

rho = 0.9  
sigma = 0.4  
sigma_in = sigma * np.sqrt(1 - rho ** 2)  

# Tolerance levels for convergence
tolv, tolr, rola = 1e-7, 1e-4, 1e-3

N = 7  
m = 3  
markov_chain = qe.markov.approximation.tauchen(N, rho, sigma_in)
Pi = markov_chain.P 
Pii = Pi.T  
log_state = markov_chain.state_values  
inv_dist = markov_chain.stationary_distributions.reshape(N,1)  
s = np.exp(log_state)  
labor = np.dot(s, inv_dist[:, 0])  
lt = len(s)  

condition = 1
 


def get_reward_space(kgrid, mu, r,w):
    """
    计算所有状态下当前资本与下一资本组合的效用
    """
    lk = len(kgrid)
    c = np.zeros((lk, lk, lt))
    u = np.zeros((lk, lk, lt))
    for t in range(lt):
        for i in range(lk):
            for j in range(lk):
                c[i, j, t] = (1 + r) * kgrid[i] + w * s[t] - kgrid[j]
                if c[i, j, t] < 0:
                    c[i, j, t] = tolv
    if mu == 1:
        u = np.log(c)
    else:
        u = (np.power(c, (1 - mu)) - 1) / (1 - mu)
    return u


def aiyagari(theta, delta, r):
    iter1 = 0  
    while True:
        iter1 += 1 
        K = labor * math.pow((r + delta) / theta, 1 / (theta - 1))  # Capital stock
        w = (1 - theta) * ((theta / (r + delta)) ** (theta / (1-theta)))  # Wage rate
        
        bound = 0
        phi = bound if r <= 0 else min(bound, w * s[0] / r)
        
        # Discretize capital for Markov process
        k_min = -phi
        k_max = 16
        inc = 0.1
        kgrid = np.arange(k_min, k_max, inc)
        lk = len(kgrid)
        
        # Initialize value function 
        u = get_reward_space(kgrid, mu,r ,w)
        V0 = np.ones((lk, lt))
        V1 = np.zeros((lk, lt))
        optim = np.zeros((lk, lt), dtype=np.int32)
        
        while np.linalg.norm(V1-V0) > tolv:
            V0 = copy.deepcopy(V1)
            for j in range(lt):
                for i in range(lk):
                    vtemp = u[i,:, j] + beta * np.dot(V0, Pii[:, j])
                    V1[i, j] = np.max(vtemp)
                    optim[i, j] = np.argmax(vtemp)
        polk = kgrid[optim.astype('int64')]
        
        # Construct transition matrix for the optimal policy
        gmat = np.zeros((lk, lk, lt))
        trans = np.zeros((lk*lt, lk*lt))
        for j in range(lt):
            for i in range(lk):
                gmat[i, optim[i, j], j] = 1
            trans[j*lk: (j+1)*lk, :] = np.kron(Pi[j, :], gmat[:, :, j])
        probst = np.ones((1, lt*lk), dtype=float) / (lt*lk)
        
        # Compute stationary distribution of capital 计算资本的稳态分布
        test = 1
        while test > 1e-5:
            probst1 = np.dot(probst, trans)
            test = np.max(np.abs(probst-probst1))
            probst = probst1
        meank = np.dot(probst, polk.T.reshape(-1, 1))
        
        # Update interest rate and check for convergence 更新利率并检查是否收敛
        rstar = theta * math.pow(meank, theta - 1) * math.pow(labor, 1 - theta) - delta
        testr = abs(r - rstar)
        testa = abs(K - meank)
        if testr < tolr:
            break  
        print()
        print(f"Mean k : {meank} ; K: {K}")
        if iter1 == 1:
            rhigh = max(r, rstar)
            rlow = min(r, rstar)
        elif meank > K:
            print(f'Current r value: {r}')
            print('Mean k > K, Reducing r')
            print()
            rhigh = r
        else:
            print(f'Current r value: {r}')
            print('Meank < K, Increasing r')
            print()
            rlow = r
        r = rhigh * 0.5 + rlow * 0.5

    print('Model solved successfully!')
    print(f"Final value of r: {r}")
    return r


### Main Results 主实验复现

In [17]:
# 调用主模型函数
theta = 0.36  # Capital's share of income
delta = 0.08  # Depreciation rate
condition = 1
r = 0.04 if condition else 0.02298754441505 
aiyagari(theta, delta,r)


Mean k : [[11.40072607]] ; K: 6.208241247715122

Mean k : [[7.45514721]] ; K: 8.169504063710336
Current r value: 0.020664397100827982
Meank < K, Increasing r


Mean k : [[9.20697512]] ; K: 7.078919626813719
Current r value: 0.03033219855041399
Mean k > K, Reducing r


Mean k : [[8.311833]] ; K: 7.592210877520186
Current r value: 0.025498297825620984
Mean k > K, Reducing r

Model solved successfully!
Final value of r: 0.02308134746322448


0.02308134746322448

### More Cases 测试更多例子

In [18]:
# 测试更多的例子
theta = 0.36  # Capital's share of income
delta = 0.08  # Depreciation rate
condition = 0
r = 0.04 if condition else 0.02298754441505 
aiyagari(theta, delta,r)



Mean k : [[7.85073408]] ; K: 7.883393105490121
Model solved successfully!
Final value of r: 0.023124538833109363


0.023124538833109363

In [19]:
# 测试更多的例子
theta = 0.36  # Capital's share of income
delta = 0.10  # Depreciation rate
condition = 1
r = 0.04 if condition else 0.02298754441505 
aiyagari(theta, delta,r)


Mean k : [[11.41426974]] ; K: 4.8793753307347885

Mean k : [[5.65940497]] ; K: 7.048818468166916
Current r value: 0.010633510129099587
Meank < K, Increasing r


Mean k : [[8.04557527]] ; K: 5.801629616148076
Current r value: 0.025316755064549792
Mean k > K, Reducing r


Mean k : [[6.73067963]] ; K: 6.375535466912307
Current r value: 0.01797513259682469
Mean k > K, Reducing r


Mean k : [[6.16304701]] ; K: 6.698327721867597
Current r value: 0.01430432136296214
Meank < K, Increasing r


Mean k : [[6.41431024]] ; K: 6.5336637593117945
Current r value: 0.016139726979893416
Meank < K, Increasing r


Mean k : [[6.56516471]] ; K: 6.453805445662083
Current r value: 0.017057429788359054
Mean k > K, Reducing r


Mean k : [[6.48091314]] ; K: 6.493533275634222
Current r value: 0.016598578384126234
Meank < K, Increasing r


Mean k : [[6.53074543]] ; K: 6.4736193809580636
Current r value: 0.016828004086242646
Mean k > K, Reducing r


Mean k : [[6.51510116]] ; K: 6.483563789617373
Current r value: 0

0.016627256596890787

In [20]:
# 测试更多的例子
theta = 0.40  # Capital's share of income
delta = 0.08  # Depreciation rate
condition = 1
r = 0.04 if condition else 0.02298754441505 
aiyagari(theta, delta,r)


Mean k : [[11.43397342]] ; K: 8.297193145803423

Mean k : [[9.28117145]] ; K: 9.665402464348663
Current r value: 0.029498457269572464
Meank < K, Increasing r


Mean k : [[10.37547102]] ; K: 8.939577325780116
Current r value: 0.03474922863478623
Mean k > K, Reducing r


Mean k : [[9.81901697]] ; K: 9.291161004674235
Current r value: 0.03212384295217935
Mean k > K, Reducing r


Mean k : [[9.55272003]] ; K: 9.47532625550747
Current r value: 0.030811150110875908
Mean k > K, Reducing r


Mean k : [[9.42676765]] ; K: 9.56960933195588
Current r value: 0.030154803690224184
Meank < K, Increasing r


Mean k : [[9.49018249]] ; K: 9.522281091519648
Current r value: 0.030482976900550046
Meank < K, Increasing r


Mean k : [[9.52802148]] ; K: 9.498757251699061
Current r value: 0.030647063505712977
Mean k > K, Reducing r

Model solved successfully!
Final value of r: 0.03056502020313151


0.03056502020313151

In [21]:
# 测试更多的例子
theta = 0.32  # Capital's share of income
delta = 0.08  # Depreciation rate
condition = 0
r = 0.04 if condition else 0.02298754441505 
aiyagari(theta, delta,r)


Mean k : [[7.54816862]] ; K: 5.909261452477154

Mean k : [[6.24278842]] ; K: 6.644740532866797
Current r value: 0.015091615259548326
Meank < K, Increasing r


Mean k : [[6.86216604]] ; K: 6.2588967152436235
Current r value: 0.01903957983729916
Mean k > K, Reducing r


Mean k : [[6.54208339]] ; K: 6.446972378737054
Current r value: 0.017065597548423746
Mean k > K, Reducing r


Mean k : [[6.38375844]] ; K: 6.544601648431685
Current r value: 0.016078606403986034
Meank < K, Increasing r


Mean k : [[6.45421862]] ; K: 6.495478871900914
Current r value: 0.01657210197620489
Meank < K, Increasing r


Mean k : [[6.49547834]] ; K: 6.471149271068622
Current r value: 0.01681884976231432
Mean k > K, Reducing r

Model solved successfully!
Final value of r: 0.016695475869259603


0.016695475869259603